## Notebook to demonstrate some CDFVI functionality

#### Standard imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
from cdfvi import funcs

#### Let's set up some cosmologies

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0 = 0., wa = 0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_w0wa = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0=-0.8, wa=-1.0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)


#### Set up the evaluation grid

In [ ]:
zmin = 0.01
zmax = 6
nzBins = 300
z_grid = np.linspace(zmin, zmax, nzBins)
z_widths = z_grid[1:] - z_grid[0:-1]
a_grid = funcs.z_to_a(z_grid)

#### Lets plot $\chi(z)$, the comoving radial distance, for three different cosmologies

In [ ]:
_ = plt.plot(z_grid, cosmo.comoving_radial_distance(a_grid)/1000., label='LCDM')
_ = plt.plot(z_grid, cosmo_w0wa.comoving_radial_distance(a_grid)/1000., label='w0wa')
_ = plt.plot(z_grid, cosmo_no_lambda.comoving_radial_distance(a_grid)/1000., label='CDM')
_ = plt.legend()
_ = plt.xlabel('z')
_ = plt.ylabel('Gpc')

In [ ]:
cosmo.h = 0.7
_ = plt.plot(z_grid, cosmo.comoving_radial_distance(a_grid)/1000., label='LCDM')
cosmo.h = 0.8
_ = plt.plot(z_grid, cosmo.comoving_radial_distance(a_grid)/1000., label='LCDM')

Now let's plot at ratio of comoving distances for different cosmologies

In [ ]:
_ = plt.plot(z_grid, cosmo_w0wa.comoving_radial_distance(a_grid)/cosmo.comoving_radial_distance(a_grid))
_ = plt.xlabel('z')
_ = plt.ylabel(r"$\chi_{wowa} / \chi_{\Lambda CDM}$")

#### Let's plot the integral $ \int_0^z \frac{1}{E(z')} dz'$ those cosmologies.

Note that the shape is exactly the same, the different that the y-axis is dimensional

In [ ]:
_ = plt.plot(z_grid[1:], funcs.j_integral_simple(cosmo, z_grid), label="LCDM")
_ = plt.plot(z_grid[1:], funcs.j_integral_simple(cosmo_w0wa, z_grid), label="w0wa")
_ = plt.plot(z_grid[1:], funcs.j_integral_simple(cosmo_no_lambda, z_grid), label='wo = 0')
_ = plt.legend()
_ = plt.xlabel('z')
_ = plt.ylabel(r"$I(z) = \int_0^z \frac{1}{E(z')} dz$")

#### Now lets plot the ratio of that integral for two different cosmologies

Note that this is exactly the same as the ratio of comoving distances

In [ ]:
_ = plt.plot(z_grid[1:], funcs.j_integral_ratio(cosmo_w0wa, cosmo, z_grid), label="LCDM")
_ = plt.xlabel("z")
_ = plt.ylabel(r"$I_{w_{0}w_{a}} / I_{\Lambda CDM}$")

In [ ]:
_ = plt.plot(z_grid, cosmo.h_over_h0(a_grid), label='LCDM')
_ = plt.plot(z_grid, cosmo_w0wa.h_over_h0(a_grid), label='w0wa')
_ = plt.plot(z_grid, cosmo_no_lambda.h_over_h0(a_grid), label='CDM')
_ = plt.ylim(0., 20.)
_ = plt.ylabel(r"$H(z) / H_{0}$")
_ = plt.legend()

In [ ]:
_ = plt.plot(z_grid, cosmo_w0wa.h_over_h0(a_grid)/cosmo.h_over_h0(a_grid), label='wowa / LCDM')
_ = plt.plot(z_grid, cosmo.h_over_h0(a_grid)/cosmo_no_lambda.h_over_h0(a_grid), label='LCDM / CDM')
_ = plt.legend()
_ = plt.ylabel(r"$H_{1}(z) / H_{2}(z)$")
_ = plt.xlabel("z")

In [ ]:
mesh = np.meshgrid(z_grid, z_grid)

In [ ]:
z_d = mesh[0]
z_s = mesh[1]
a_d = funcs.z_to_a(z_d)
a_s = funcs.z_to_a(z_s)
chi_d = cosmo_no_lambda.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
chi_s = cosmo_no_lambda.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
r_tilde = chi_s / (a_d * (chi_s - chi_d))

In [ ]:
r = funcs.r_tilde_flat(cosmo, mesh[0], mesh[1])
r1 = funcs.r_tilde_flat(cosmo_no_lambda, mesh[0], mesh[1])
rw = funcs.r_tilde_flat(cosmo_w0wa, mesh[0], mesh[1])

chi_r = funcs.chi_ratio(cosmo, mesh[0], mesh[1])
chi_r1 = funcs.chi_ratio(cosmo_no_lambda, mesh[0], mesh[1])
chi_rw = funcs.chi_ratio(cosmo_w0wa, mesh[0], mesh[1])

R_r = funcs.r_flat(cosmo, mesh[0], mesh[1])
R_r1 = funcs.r_flat(cosmo_no_lambda, mesh[0], mesh[1])
R_rw = funcs.r_flat(cosmo_w0wa, mesh[0], mesh[1])


In [ ]:
_ = plt.imshow(R_r, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{ds} = \chi_{d}/\chi_{s}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')

In [ ]:
_ = plt.imshow(chi_r, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{ds} = \chi_{d}/\chi_{s}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')

In [ ]:
_ = plt.imshow(chi_r/chi_r1, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{\Lambda CDM} / R_{CDM}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM / CDM$')

In [ ]:
_ = plt.imshow(chi_rw/chi_r, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$R_{w_{0}w_{a}} / R_{\Lambda CDM}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$w_{0}w_{a} / \Lambda CDM$')

In [ ]:
_ = plt.imshow(r, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$')

In [ ]:
_ = plt.imshow(r1, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'CDM Only')

In [ ]:
_ = plt.imshow(rw, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$w_{0}w_{a}$')

In [ ]:
_ = plt.imshow(r/r1, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"R $D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$\Lambda CDM$ / CDM Only')

In [ ]:
_ = plt.imshow(rw/r, origin='lower', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"R $D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm d}$')
_ = plt.ylabel(r'$z_{\rm s}$')
_ = plt.title(r'$w_{0}w_{a} / \Lambda CDM$')